# Data Visualization

In [ ]:
import pandas as pd
from pathlib import Path

xbd_path = 'datasets/xbd'
subsets = ('/train_bldgs/', '/hold_bldgs/', '/test_bldgs/', '/tier3_bldgs/')
disaster_folders = os.listdir(xbd_path + subsets[0])

i_subset = 0
i_disaster = 5

print(list(Path(xbd_path + subsets[i_subset] + disaster_folders[i_disaster]).glob('*.csv*'))[0])
labels = pd.read_csv(list(Path(xbd_path + subsets[i_subset] + disaster_folders[i_disaster]).glob('*.csv*'))[0])
labels.columns = ['name', 'xcoords', 'ycoords', 'long', 'lat', 'class']
zone = lambda row: '_'.join(row['name'].split('_', 2)[:2])
labels['zone'] = labels.apply(zone, axis=1)
labels['zone'].value_counts()
#labels['zone'].value_counts()[labels['zone'].value_counts()==1].index.tolist()

In [ ]:
from visualization import plot_on_image

plot_on_image(labels, subsets[i_subset], 'mexico-earthquake_00000098')

In [ ]:
from visualization import plot_on_map

plot_on_map(labels, mapbox=False)

In [ ]:
import matplotlib.pyplot as plt
from visualization import CmapString

cmap = CmapString(palette='viridis', domain=labels['zone'].values)

plt.figure(figsize=(12,8))
for _, row in labels.iterrows():
    plt.scatter(row['xcoords'], row['ycoords'], label=row['zone'], color=cmap.color(row['zone']))
plt.axis('off')
plt.show()

## Polygon Visualization

In [ ]:
from visualization import plot_image

plot_image("D:/thesis/datasets/xbd/train/labels/mexico-earthquake_00000098_post_disaster.json")

---

# Image Reshaping

In [ ]:
from PIL import Image
from torchvision.transforms import ToTensor

tr = ToTensor()

image_pre = Image.open("D:/thesis/datasets/socal-fire_00000012_pre_disaster_2.png").resize((128,128))
image_post = Image.open("D:/thesis/datasets/socal-fire_00000124_post_disaster_4.png").resize((128,128))

In [ ]:
import torch

image_pre = tr(image_pre)
image_post = tr(image_post)
images = torch.cat((image_pre, image_post),0)

In [ ]:
imgs = images.flatten().unsqueeze(dim=0)

In [ ]:
newimgs = imgs.reshape((-1, 6, 128, 128))

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(newimgs[0,:3,:,:].permute(1,2,0))
plt.show()

In [ ]:
plt.imshow(newimgs[0,3:,:,:].permute(1,2,0))
plt.show()

---

# Graph Generation

In [ ]:
import os
from pathlib import Path
import json
import pandas as pd

path = 'datasets/xbd/train_bldgs/'
disaster_folders = os.listdir(path)
disaster = 'socal-fire'

labels = pd.read_csv(list(Path(path + disaster).glob('*.csv*'))[0], index_col=0)
labels.drop(columns=['long','lat'], inplace=True)
zone = lambda row: '_'.join(row.name.split('_', 2)[:2])
labels['zone'] = labels.apply(zone, axis=1)

processed_files = []
zones = labels['zone'].value_counts()[labels['zone'].value_counts()>1].index.tolist()
for zone in zones:
     if not ((labels[labels['zone'] == zone]['class'] == 'un-classified').all() or \
            (labels[labels['zone'] == zone]['class'] != 'un-classified').sum() == 1):
        processed_files.append(f'{zone}.pt')

len(processed_files)

In [ ]:
for zone in [zones[0]]:
    if (labels[labels['zone'] == zone]['class'] == 'un-classified').all() or \
    (labels[labels['zone'] == zone]['class'] != 'un-classified').sum() == 1:
        continue
    print(f'Building {zone}...')
    list_pre_images = list(map(str, Path(path + disaster).glob(f'{zone}_pre_disaster*')))
    list_post_images = list(map(str, Path(path + disaster).glob(f'{zone}_post_disaster*')))
    coords = []

    for pre_image_file, post_image_file in zip(list_pre_images, list_post_images):
        
        annot = labels.loc[os.path.split(post_image_file)[1],'class']
        if annot == 'un-classified':
            continue
        coords.append((labels.loc[os.path.split(post_image_file)[1],'xcoord'],
                        labels.loc[os.path.split(post_image_file)[1],'ycoord']))

---

# Graph Connectivity

In [ ]:
import torch
import networkx as nx
from torch_geometric.utils import to_networkx

data = torch.load('datasets/socal-fire_00000908.pt')
data.pos = torch.tensor(coords)
datax = to_networkx(data)
pos = dict(enumerate(data.pos.numpy()))
pos = {node: (x,-y) for (node, (x,y)) in pos.items()}
nx.draw_networkx(datax, pos=pos, arrows=False, with_labels=False, node_size=20, node_color='red')

In [ ]:
from visualization import plot_on_image
plot_on_image(labels, 'train', 'socal-fire_00000908')

In [ ]:
from torch_geometric.transforms import Compose, Delaunay, FaceToEdge
trans = Compose([Delaunay(), FaceToEdge()])
datad = trans(data)
datad

In [ ]:
datadx = to_networkx(datad)
nx.draw_networkx(datadx, pos=pos, arrows=False, with_labels=False, node_size=20, node_color='red')

---

In [ ]:
from model import SiameseEncoder

enc = SiameseEncoder(diff=False)

In [ ]:
enc.get_output_shape()

In [ ]:
import torch

data = torch.load('datasets/socal-fire_00001135.pt')
data